In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

### Simple AI Assitant

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings= GoogleGenerativeAIEmbeddings(model='models/embedding-001')
llm= ChatGoogleGenerativeAI(model='gemini-1.5-pro')

In [4]:
while True:
    question= input('type your question. if you want to quit the chat type quit')
    if question != 'quit':
        print(llm.invoke(question).content)
    else:
        print('good bye, take care yourself')
        break

Hello Manirathinam! 

It's nice to meet you. What can I do for you today? 

## AI Hallucination in RAG: A Risky Reality

**RAG (Retrieval Augmented Generation)** is a powerful technique in AI that combines the strengths of information retrieval and text generation. It works by retrieving relevant information from a knowledge base and using it to generate more comprehensive and informative responses. 

However, this reliance on external data makes RAG systems susceptible to a phenomenon called **AI hallucination**.

**What is AI Hallucination?**

In the context of RAG, AI hallucination occurs when the generated text:

* **Presents fabricated information as facts:** This can involve inventing statistics, misrepresenting historical events, or creating entirely fictional entities.
* **Draws inaccurate conclusions from retrieved data:** The system might misinterpret the information, leading to logically flawed or misleading statements.
* **Generates irrelevant or nonsensical content:**  Eve

### Adding Chat History to AI Assitant

In [19]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage,HumanMessage

In [53]:
store={}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]= InMemoryChatMessageHistory()
        return store[session_id]

config= {'configurable':{'session_id':'abc123'}}

In [54]:
llm_with_memory= RunnableWithMessageHistory(llm, get_session_history)

llm_with_memory.invoke([HumanMessage(content="Hi! My name is Manirathinam")], config= config).content

"Hi Manirathinam, it's nice to meet you! 👋 \n\nWhat can I do for you today? 😊 \n"

In [56]:
llm_with_memory.invoke([HumanMessage(content="what is 2+5?")],config=config).content

AttributeError: 'NoneType' object has no attribute 'messages'

In [28]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi!, I'm Manirathinam", additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Manirathinam! \n\nIt's nice to meet you. What can I do for you today? \n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-9900c802-e856-4c1a-abdd-36920695fef7-0', usage_metadata={'input_tokens': 10, 'output_tokens': 24, 'total_tokens': 34})])}

### Basic RAG

In [31]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

### Reading the txt files from source directory

loader = DirectoryLoader('../data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

###  BGE Embddings

'''from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
'''

### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [33]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [35]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

question ="what is llama3? can you highlight 3 important points?"
print(retrieval_chain.invoke(question))

Based on the provided text snippets, here's what we can gather about Llama 3:

**What is Llama 3?**

Llama 3 is a language model. While the provided text doesn't explicitly define it, we can infer this from the mentions of "parameter version" and its use in services. 

**Important Points:**

1. **Release:** Llama 3 was likely released in April 2024. This is suggested by the fragment  "Llama 3 released in April 2024."
2. **Model Versions:** There are different versions of Llama 3, including at least one with 8 billion parameters.  ("...the 8B parameter version of Llama 3...")
3. **Accessibility & Usage:**  Llama 3 is used in various services and some versions were made accessible to the public. ("Both services use a Llama 3 model." and "versions of Llama were made accessible outside") 



## Let's Start with Tools and Agents

### Wikipedia

In [36]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper= WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=100)
tool= WikipediaQueryRun(api_wrapper=api_wrapper)

tool.name

'wikipedia'

In [37]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [38]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [39]:
print(tool.run({'query':'LTTE Prabhakaran'}))

Page: Velupillai Prabhakaran
Summary: Velupillai Prabhakaran (; Tamil: வேலுப்பிள்ளை பிரபாகரன்; [ˈʋeː


### Youtube search tool

In [44]:
from langchain_community.tools import YouTubeSearchTool

utubetool=YouTubeSearchTool()

utubetool.name

'youtube_search'

In [45]:
utubetool.description

'search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional'

In [46]:
utubetool.run("sachin tendulkar")

ModuleNotFoundError: No module named 'youtube_search'

### Tavily Search (langchain browser)

In [48]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool3 = TavilySearchResults()
tool3.invoke({"query": "What happened in the 2024 T20 world cup"})


[{'url': "https://en.wikipedia.org/wiki/2024_ICC_Men's_T20_World_Cup_final",
  'content': "The 2024 ICC Men's T20 World Cup final was a Twenty20 International cricket match played at Kensington Oval in Bridgetown, Barbados on 29 June 2024 to determine the winner of the 2024 ICC Men's T20 World Cup. [1] [2] It was played between South Africa and India.[3]India defeated South Africa by 7 runs to win their second T20 World Cup title. [4] Virat Kohli was named Player of the Match for ..."},
 {'url': 'https://www.espncricinfo.com/story/t20-world-cup-2024-final-ind-vs-sa-history-weighs-heavy-as-south-africa-die-another-death-1441561',
  'content': 'T20 World Cup 2024 - a zero for Bumrah, and other curious numbers ... This was the first time they had reached this stage of a men\'s World Cup in either format, ... "Things happened very quickly ...'},
 {'url': 'https://www.aljazeera.com/sports/liveblog/2024/10/13/live-india-vs-australia-icc-womens-t20-world-cup-2024-group-a-match',
  'content': 

### Agents

In [51]:
from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [52]:
tool=load_tools(['wikipedia'], llm=llm)
agent= initialize_agent(tool, llm, agent= AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent.run('what is current GDP of India')

C:\Users\manir\AppData\Local\Temp\ipykernel_1100\1345114947.py:2: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent= initialize_agent(tool, llm, agent= AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
C:\Users\manir\AppData\Local\Temp\ipykernel_1100\1345114947.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run('what is current GDP of India')




> Entering new AgentExecutor chain...
Question: what is current GDP of India
Thought: I need to search for India's GDP on Wikipedia. 
Action: wikipedia
Action Input: GDP of India
Observation: Page: Economy of India
Summary: The economy of India is a developing mixed economy with a notable public sector in strategic sectors. It is the world's fifth-largest economy by nominal GDP and the third-largest by purchasing power parity (PPP); on a per capita income basis, India ranked 136th by GDP (nominal) and 125th by GDP (PPP). From independence in 1947 until 1991, successive governments followed the Soviet model and promoted protectionist economic policies, with extensive Sovietization, state intervention, demand-side economics, natural resources, bureaucrat driven enterprises and economic regulation. This is characterised as dirigism, in the form of the Licence Raj. The end of the Cold War and an acute balance of payments crisis in 1991 led to the adoption of a broad economic liberalisati

'I cannot answer the question using the provided context.'